<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/fine_tuning_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Embedding Models

In the following Notebook we will be exploring one of the most powerful techniques to take your single-domain RAG pipelines to the next level...

## Task 1: Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API key!

### Nest Asyncio

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

In [ ]:
!pip install -qU langchain_mistralai langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters

In [ ]:
!pip install -qU faiss-cpu unstructured==0.15.7 python-pptx==1.0.2 nltk==3.9.1 openparse[ml] html2text langchain langchain_groq langchain_huggingface faiss-cpu langchain-pinecone pinecone-notebooks langchain_mistralai
!mkdir -p data
!wget https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico -O "data/ppc_2018.pdf"

--2024-11-02 22:27:35--  https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico
Resolving www.uffs.edu.br (www.uffs.edu.br)... 200.135.49.107
Connecting to www.uffs.edu.br (www.uffs.edu.br)|200.135.49.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1398975 (1.3M) [application/pdf]
Saving to: ‘data/ppc_2018.pdf’

data/ppc_2018.pdf   100%[===================>]   1.33M   437KB/s    in 3.1s    

2024-11-02 22:27:40 (437 KB/s) - ‘data/ppc_2018.pdf’ saved [1398975/1398975]



In [ ]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')
  MISTRAL_AI_KEY    = userdata.get('MISTRAL_AI_KEY')
  GROQ_API_KEY2     = userdata.get('GROQ_API_KEY2')
  MISTRAL_AI_KEY2   = userdata.get('MISTRAL_AI_KEY2')


settings = Settings()

## Task 2: Loading Data

The data can be found in [this GitHub repo](https://github.com/AI-Maker-Space/DataRepository/tree/main/high-performance-rag).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/My\ Drive/tcc/PPC_2024.pdf data/ppc_2024.pdf

In [ ]:
from langchain_core.load import dumpd, dumps, load, loads
import json
import time
import hashlib
import os


def save_document_locally(documents, version):

  output_file_name = f'ppc{version}_documents.json'
  json_string = dumps(documents, pretty=True, ensure_ascii=False)
  with open(output_file_name, "w", encoding="utf-8") as file:
      file.write(json_string)

def load_document_locally(version):
  output_file_name = f'ppc{version}_documents.json'
  with open(output_file_name, "r") as fp:
      doc = json.load(fp)
  return load(doc)


def save_dict_to_json_on_drive(documents, drive_folder_path, drive_file_name):

    os.makedirs(drive_folder_path, exist_ok=True)
    timestamp = str(int(time.time()))
    hash_suffix = hashlib.md5(timestamp.encode()).hexdigest()[:6]  # Gera um hash curto com 6 caracteres

    drive_file_name = f'{hash_suffix}_{drive_file_name}'
    drive_path = f'{drive_folder_path}/{os.path.basename(drive_file_name)}'

    json_string = dumpd(documents)
    print(json_string)
    with open(drive_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_string, json_file, ensure_ascii=False)
    print(f'File saved to: {drive_path}')
    return drive_path

In [ ]:
import openparse
import html2text
from langchain_core.documents import Document
from uuid import uuid4


def parse_document_to_documents(file_path, source_name):
    # Parse the document using OpenParse
    parser = openparse.DocumentParser(
                                      table_args = {"parsing_algorithm": "pymupdf", "table_output_format": "markdown"}
                                      )
    document = parser.parse(file_path)

    documents = []
    ids = []

    for node in document.nodes:

        html_content = node.text

        if html_content:
            id = str(uuid4())
            ids.append(id)
            page = node.bbox[0].page + 1
            document_chunk = Document(
                metadata={'source': source_name,
                          'id': str(uuid4()),
                          'page': page},
                page_content=html_content
            )
            documents.append(document_chunk)

    return documents, ids, document


In [ ]:
documents_2018, ids_2018, nodes_2018 = parse_document_to_documents('data/ppc_2018.pdf','ppc2018')
documents_2024, ids_2024, nodes_2024 = parse_document_to_documents('data/ppc_2024.pdf','ppc2024')

In [ ]:
from sklearn.model_selection import train_test_split

training_split_documents = documents_2024
val_split_documents, test_split_documents = train_test_split(documents_2018, test_size=0.5, random_state=42)


In [ ]:
total = len(training_split_documents)
print(f'Treinamento: {len(training_split_documents)} documentos ({len(training_split_documents)/total:.1%})')
print(f'Validação: {len(val_split_documents)} documentos ({len(val_split_documents)/total:.1%})')
print(f'Teste: {len(test_split_documents)} documentos ({len(test_split_documents)/total:.1%})')

Treinamento: 510 documentos (100.0%)
Validação: 206 documentos (40.4%)
Teste: 207 documentos (40.6%)


In [ ]:
# # Divide os dados em três partes de uma vez
# from sklearn.model_selection import train_test_split
# training_documents = documents_2018 + documents_2024
# training_split_documents, temp_split = train_test_split(training_documents, test_size=0.3, random_state=42)
# val_split_documents, test_split_documents = train_test_split(temp_split, test_size=0.5, random_state=42)

# total = len(training_documents)
# print(f'Treinamento: {len(training_split_documents)} documentos ({len(training_split_documents)/total:.1%})')
# print(f'Validação: {len(val_split_documents)} documentos ({len(val_split_documents)/total:.1%})')
# print(f'Teste: {len(test_split_documents)} documentos ({len(test_split_documents)/total:.1%})')

Treinamento: 646 documentos (70.0%)
Validação: 138 documentos (15.0%)
Teste: 139 documentos (15.1%)


## Task 3: Constructing a Fine-tuning Dataset

Using the nodes we created above, we can finally start constructing a fine-tuning dataset utilizing OpenAI's `gpt-4o-mini` (released [today](https://openai.com/index/gpt-4o-mini-advancing-cost-efficient-intelligence/)).

The basic idea here is straightforward enough:

1. We look at a document
2. We generate questions that could be answered by that node

This gives us a number of question/context pairs that we can use to fine-tune our Embeddings model.

In [ ]:
from langchain_mistralai import ChatMistralAI

qa_chat_model = ChatMistralAI(model='open-mixtral-8x7b',
                    temperature=1,
                    max_retries=2,
                    api_key=settings.MISTRAL_AI_KEY2)

We'll create a simple Question Generation prompt to query `gpt-4o-mini` to generate Questions for each retrieved context.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
import tqdm
import time

qa_prompt = """\
Sua tarefa é gerar perguntas que simulem dúvidas reais de alunos sobre o Projeto Pedagógico do Curso (PPC) \
de Ciência da Computação. Gere {n_questions} perguntas que possam ser respondidas EXCLUSIVAMENTE \
com as informações presentes no contexto fornecido. Responda EXCLUSIVAMENTE em português.

IMPORTANTE: Gere APENAS perguntas que podem ser respondidas completamente usando o texto fornecido. \
Não crie perguntas que exijam informações externas ou que não estejam explicitamente mencionadas no contexto.

Diretrizes para geração:
1. Analise cuidadosamente o conteúdo do contexto antes de gerar as perguntas
2. Cada pergunta deve ter sua resposta claramente presente no texto
3. Use linguagem natural, como alunos realmente perguntariam
4. Evite linguagem excessivamente formal ou perguntas extensas; seja direto e objetivo
5. Varie a forma de perguntar sobre as informações disponíveis
6. Alterne entre perguntas diretas e indiretas sobre o mesmo conteúdo
7. Gere perguntas alinhadas com o interesse de alunos

Tópicos comumente abordados:
- Estrutura curricular (grade, pré-requisitos, optativas, ementa de disciplinas)
- Carga horária (total, por disciplina, atividades complementares)
- Políticas e regulamentos (frequência, avaliações, jubilamento)
- Atividades acadêmicas (estágio, TCC, projetos, monitoria)
- Diretrizes institucionais (transferência, trancamento, aproveitamentos)
- Infraestrutura e recursos disponíveis
- Procedimentos administrativos

Variações a incluir:
- Reformulações de informações explícitas no texto
- Diferentes formas de questionar o mesmo conteúdo
- Perguntas com diferentes níveis de especificidade sobre as informações disponíveis
- Combinações de informações relacionadas presentes no texto

Nota: Evite verbosidade e não use extensões desnecessárias como "no curso de Ciência da Computação" ou "no plano pedagógico do curso".

Exemplos do tom esperado (considere apenas se estas informações estiverem no contexto):
- "Qual é a carga horária total?"
- "Quais são as disciplinas obrigatórias do primeiro semestre?"
- "Como funciona o TCC?"

Formato de saída:
PERGUNTA #1,PERGUNTA #2,PERGUNTA #3,PERGUNTA #4
...

Contexto: {context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)
output_parser = CommaSeparatedListOutputParser()
question_generation_chain = qa_prompt_template | qa_chat_model | output_parser




We'll create a simple chain to query the LLM!

There's a lot going on in this function - let's take a deeper look:

1. First, we provide a list of documents and a number of questions
2. We, for each document in our list, generate `n_questions` of questions.
3. We then associate those questions and contexts via a `UUID`.

> NOTE: The reason we're doing this `UUID` association is for ease of use later in the notebook.

In [ ]:
import tqdm
import uuid
import time

def create_questions(documents, n_questions):
    questions = []
    relevant_docs = {}
    for document in tqdm.tqdm(documents):
        attempt = 0
        while attempt < 3:
            try:
                time.sleep(1)
                questions_generated = question_generation_chain.invoke({"context": document.page_content, "n_questions": n_questions})
                print(questions_generated)
                print(len(questions_generated))
                print(type(questions_generated))

                for question in questions_generated:
                    question_text = question.strip()
                    if question_text:
                        question_id = str(uuid.uuid4())
                        questions.append(question_text)
                        relevant_docs[question_id] = [document.metadata["id"]]
                break
            except Exception as e:
                print(f"Erro ao gerar perguntas: {e}. Tentando novamente em 15 segundos...")
                time.sleep(15)
                attempt += 1
    return questions, relevant_docs


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
import tqdm
import uuid
import time

qa_prompt = """\
Sua tarefa é gerar perguntas que simulem dúvidas reais de alunos sobre o Projeto Pedagógico do Curso (PPC) \
de Ciência da Computação. Gere {n_questions} perguntas que possam ser respondidas EXCLUSIVAMENTE \
com as informações presentes no contexto fornecido. Gere as perguntas EXCLUSIVAMENTE em português.

IMPORTANTE: Gere APENAS perguntas que podem ser respondidas completamente usando o texto fornecido. \
Não crie perguntas que exijam informações externas ou que não estejam explicitamente mencionadas no contexto.

Diretrizes para geração:
1. Analise cuidadosamente o conteúdo do contexto antes de gerar as perguntas
2. Cada pergunta deve ter sua resposta claramente presente no texto
3. Use linguagem natural, como alunos realmente perguntariam
4. Evite linguagem excessivamente formal ou perguntas extensas; seja direto e objetivo
5. Varie a forma de perguntar sobre as informações disponíveis
6. Alterne entre perguntas diretas e indiretas sobre o mesmo conteúdo
7. Gere perguntas alinhadas com o interesse de alunos
8. Exclusivamente perguntas em português brasileiro.

Tópicos comumente abordados:
- Estrutura curricular (grade, pré-requisitos, optativas, ementa de disciplinas)
- Carga horária (total, por disciplina, atividades complementares)
- Políticas e regulamentos (frequência, avaliações, jubilamento)
- Atividades acadêmicas (estágio, TCC, projetos, monitoria)
- Diretrizes institucionais (transferência, trancamento, aproveitamentos)
- Infraestrutura e recursos disponíveis
- Procedimentos administrativos

Variações a incluir:
- Reformulações de informações explícitas no texto
- Diferentes formas de questionar o mesmo conteúdo
- Perguntas com diferentes níveis de especificidade sobre as informações disponíveis
- Combinações de informações relacionadas presentes no texto

**Nota:** Evite verbosidade e não use extensões desnecessárias como "no curso de Ciência da Computação" ou "no plano pedagógico do curso".

Exemplos do tom esperado (considere apenas se estas informações estiverem no contexto):
- "Qual é a carga horária total?"
- "Quais são as disciplinas obrigatórias do primeiro semestre?"
- "Como funciona o TCC?"

Formato de saída:
1. Primeira pergunta aqui
2. Segunda pergunta aqui
3. Terceira pergunta aqui
...

Contexto: {context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

def create_questions(documents, n_questions, qa_chat_model):
    questions = {}
    relevant_docs = {}

    for document in tqdm.tqdm(documents):
        attempt = 0
        while attempt < 3:  # Try up to 3 times
            try:
                time.sleep(1)
                questions_generated = qa_chat_model.invoke(
                    qa_prompt_template.format(
                        context=document.page_content,
                        n_questions=n_questions
                    )
                )


                # Extrair e limpar a lista de questões
                questions_list = [
                    q.strip()
                    for q in questions_generated.content.split('\n')
                    if q.strip() and any(q.strip().startswith(str(i) + ".") for i in range(1, n_questions + 1))
                ]
                print(len(questions_list))
                print(questions_generated)
                # Processar cada questão da lista
                for question in questions_list:
                    question_id = str(uuid.uuid4())
                    # Remove o número e o ponto do início
                    clean_question = "".join(question.split(".")[1:]).strip()
                    questions[question_id] = clean_question
                    relevant_docs[question_id] = [document.metadata["id"]]

                break  # If successful, exit the retry loop

            except Exception as e:
                print(f"Error generating questions: {e}. Retrying in 15 seconds...")
                time.sleep(15)
                attempt += 1

                if attempt == 3:
                    print(f"Failed to generate questions for document after 3 attempts")

    return questions, relevant_docs

We'll use the function to generate training, validation, and test data.

In [ ]:
training_questions, training_relevant_contexts = create_questions(training_split_documents, 4, qa_chat_model)

  0%|          | 1/510 [00:02<24:43,  2.91s/it]

4
content='1. Em quantos estados brasileiros a Universidade Federal da Fronteira Sul está localizada?\n2. Qual é a cidade sede da universidade?\n3. Quantos *campi* possui a universidade em cada um dos estados onde está presente?\n4. Em quais cidades, além da sede, a universidade possui *campi* no estado de Santa Catarina?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 930, 'total_tokens': 1022, 'completion_tokens': 92}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-d0fa4ee6-a979-45d5-9436-34b71416deab-0' usage_metadata={'input_tokens': 930, 'output_tokens': 92, 'total_tokens': 1022}


  0%|          | 2/510 [00:05<23:53,  2.82s/it]

4
content='1. Quem é o Pro-Reitor responsável pelo setor de Graduação no nosso curso?\n2. Além da grade obrigatória, o curso oferece disciplinas optativas? Se sim, quais são as áreas abrangidas por essas disciplinas?\n3. Como o processo de avaliação dos alunos é conduzido no curso de Ciência da Computação?\n4. Existe algum procedimento específico para realização do Trabalho de Conclusão de Curso (TCC) no curso?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1008, 'total_tokens': 1145, 'completion_tokens': 137}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-071c00fe-2afe-48c9-83e3-76acff60a804-0' usage_metadata={'input_tokens': 1008, 'output_tokens': 137, 'total_tokens': 1145}


  1%|          | 3/510 [00:09<26:25,  3.13s/it]

...

4
content='1. Quanto de créditos as disciplinas de Cálculo I e Geometria Analítica valem no currículo atual?\n2. Em comparação com a matriz curricular de 2018, como é chamada a disciplina de Libras atualmente?\n3. Para as disciplinas de cálculo, quantos créditos são atribuídos às suas respectivas contrapartes nos CCRs Matriz 2023?\n4. A disciplina de Probabilidade e Estatística mantém a mesma quantidade de créditos em ambas as matrizes curriculares comparadas?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1113, 'total_tokens': 1269, 'completion_tokens': 156}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-266290a3-06e7-4fce-b6df-31730ee884d6-0' usage_metadata={'input_tokens': 1113, 'output_tokens': 156, 'total_tokens': 1269}


100%|█████████▉| 509/510 [38:04<00:03,  3.47s/it]

4
content='1. Em que situações um componente curricular do curso de Ciência da Computação pode ser validado por equivalência?\n2. Que figura no histórico escolar do aluno quando um componente curricular é validado por equivalência?\n3. Como é calculada a nota de registro quando mais de um componente curricular da matriz de origem é usado para validar um componente da matriz de destino?\n4. De acordo com o parágrafo único, como os componentes curriculares equivalentes são registrados nas matrículas dos alunos?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 949, 'total_tokens': 1093, 'completion_tokens': 144}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-1bb4692e-bb88-4078-b7f7-75acbe9c9ff4-0' usage_metadata={'input_tokens': 949, 'output_tokens': 144, 'total_tokens': 1093}


100%|██████████| 510/510 [38:07<00:00,  4.49s/it]

4
content='1. Quantos créditos correspondem às disciplinas "Introdução à computação", "Introdução ao desenvolvimento web" e "Iniciação à prática de extensão" no curso?\n2. Além do "Projeto de Integração de Extensão", quais outras matérias do currículo não têm equivalência entre as matrizes do curso de Ciência da Computação?\n3. Qual é o número total de créditos dos componentes curriculares listados no contexto?\n4. Em relação às matérias obrigatórias do curso, quais são os creditos correspondentes à disciplina "Projeto de Integração de Extensão"?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 978, 'total_tokens': 1147, 'completion_tokens': 169}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-d8decd53-e9fb-4993-9ab5-848fb1ec835e-0' usage_metadata={'input_tokens': 978, 'output_tokens': 169, 'total_tokens': 1147}


In [ ]:
val_questions, val_relevant_contexts = create_questions(val_split_documents, 4, qa_chat_model)

  0%|          | 1/206 [00:03<10:58,  3.21s/it]

4
content='1. Quantos créditos são necessários para se matricular em "Tópicos Especiais em Computação II"?\n2. Quais são os pré-requisitos em horas para se cursar "Tópicos Especiais em Computação III" no turno noturno?\n3. Em quantas horas semanais é oferecida a disciplina "Tópicos Especiais em Computação IV"?\n4. Para se inscrever em "Tópicos Especiais em Computação XV", quais as respectivas obrigações de créditos no turno diurno?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1842, 'total_tokens': 1990, 'completion_tokens': 148}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-22453d5b-5c07-42dc-ab05-9faee3dbf7b6-0' usage_metadata={'input_tokens': 1842, 'output_tokens': 148, 'total_tokens': 1990}


  1%|          | 2/206 [00:06<11:11,  3.29s/it]

4
content='1. Quais são os requisitos que cada documento anexo deve atender para ser incluído na compilação de acordo com o § 3º?\n2. O que é necessário incluir em cada texto de contextualização para os documentos anexos, de acordo com o § 4º?\n3. Como é determinada a pontuação mínima necessária para a soma de documentos anexos, de acordo com o § 5º?\n4. De acordo com o § 5º, é possível incluir documentos de mesma natureza com diferentes pontuações na compilação?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1163, 'total_tokens': 1319, 'completion_tokens': 156}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-c5fe8418-c26a-4c28-a92d-3b9fb09f76d2-0' usage_metadata={'input_tokens': 1163, 'output_tokens': 156, 'total_tokens': 1319}


  1%|▏         | 3/206 [00:09<10:56,  3.23s/it]

...

4
content='1. Quem é responsável pelo tratamento técnico do material bibliográfico nas bibliotecas da UFFS?\n2. Quais órgãos estão integrados e atuam de forma sistêmica nas bibliotecas da UFFS?\n3. Para além do tratamento técnico do material bibliográfico, o que mais cabe à responsabilidade da Divisão de Bibliotecas (DBIB)?\n4. Quais serviços à comunidade devem ser garantidos em cada unidade das bibliotecas dos diferentes *campi*?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1130, 'total_tokens': 1268, 'completion_tokens': 138}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-a84ac548-1d02-4177-bba5-56e4c940b22f-0' usage_metadata={'input_tokens': 1130, 'output_tokens': 138, 'total_tokens': 1268}


100%|█████████▉| 205/206 [15:27<00:05,  5.71s/it]

4
content='1. Quanto vale a pontuação para um relatório de participação em uma etapa nacional de Maratona de Programação, descrevendo e discutindo um dos problemas da prova?\n2. Qual é a pontuação atribuída a um relatório de participação em um projeto de Iniciação Científica institucionalizado, endossado pelo professor responsável e com duração mínima de um ano?\n3. É permitido que mais de um aluno apresente um relatório com análise do mesmo problema em uma etapa regional de Maratona de Programação?\n4. Quais são os requisitos para a pontuação de 30, relacionada à participação como gerente de equipe em um projeto conduzido por grupo(s) institucionalizado(s) do curso?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1340, 'total_tokens': 1545, 'completion_tokens': 205}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-14afba23-7d1b-4574-ae93-0491b6337687-0' usage_metadata={'input_tokens': 1340, 'output_tokens': 205, 'total_tokens': 1545}


100%|██████████| 206/206 [15:30<00:00,  4.52s/it]

4
content='1. Quantas horas semanais de aula teórica tem a disciplina de "Pesquisa e ordenação de dados"?\n2. Quem é o docente responsável pela disciplina de "Linguagens formais e autômatos"?\n3. Em que percentual a disciplina de "Sistemas operacionais" é dividida entre sala de aula e laboratório?\n4. Quantos créditos tem a disciplina "Iniciação à prática científica" e é realizada totalmente em sala de aula?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1848, 'total_tokens': 1981, 'completion_tokens': 133}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-d56d6ff1-145c-4a57-b7be-b2d18b9507f4-0' usage_metadata={'input_tokens': 1848, 'output_tokens': 133, 'total_tokens': 1981}


In [ ]:
test_questions, test_relevant_contexts = create_questions(test_split_documents, 4, qa_chat_model)

  0%|          | 1/207 [00:02<09:04,  2.64s/it]

4
content='1. Em que momentos serão definidos os critérios e formas de avaliação do TCC I e II?\n2. O processo de avaliação do TCC I e II está previsto para ser especificado em que partes do curriculo?\n3. Após a homologação, onde podemos encontrar os critérios de avaliação para o TCC I e II?\n4. As formas de avaliação do TCC I e II serão descritas em seus respectivos planos de ensino?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 884, 'total_tokens': 1012, 'completion_tokens': 128}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-03e13e61-a4c3-4a4f-b15e-a19d6ae2deae-0' usage_metadata={'input_tokens': 884, 'output_tokens': 128, 'total_tokens': 1012}


  1%|          | 2/207 [00:06<10:32,  3.09s/it]

4
content='1. Quais condições devem ser cumpridas para que a participação em um projeto de empreendedorismo ou inovação seja reconhecida pelo Colegiado do Curso?\n2. Em relação a um projeto de Empreendedorismo ou Inovação, quais são as etapas e respectivos documentos entregáveis esperados no relatório?\n3. Qual é o prazo mínimo de duração para a participação em um projeto institucionalizado do curso, como uma empresa júnior ou grupo de extensão?\n4. Quais são os passos necessários para que o produto desenvolvido em um projeto de empreendedorismo ou inovação seja entregue a um cliente ou usuário externo?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1204, 'total_tokens': 1397, 'completion_tokens': 193}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-28093551-fd8b-4426-a251-220607ca737a-0' usage_metadata={'input_tokens': 1204, 'output_tokens': 193, 'total_tokens': 1397}


  1%|▏         | 3/207 [00:09<11:03,  3.25s/it]

...

4
content='1. Quantos créditos são atribuídos ao componente curricular GEX607 - ÁLGEBRA LINEAR?\n2. Quais são os objetivos da disciplina ÁLGEBRA LINEAR?\n3. Em que edição está a referência básica Álgebra linear com aplicações de Anton, H. e Rorres, C.?\n4. O livro Álgebra linear e aplicações de Callioli, C.; Domingues, H.; e Costa, R. é considerado uma referência complementar para a disciplina ÁLGEBRA LINEAR?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1660, 'total_tokens': 1803, 'completion_tokens': 143}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-c8b14fdc-f7d6-4727-aa13-6773611138ac-0' usage_metadata={'input_tokens': 1660, 'output_tokens': 143, 'total_tokens': 1803}


100%|█████████▉| 206/207 [15:28<00:03,  3.28s/it]

4
content='1. Quantos créditos e horas são dedicados ao estudo da disciplina de Computação Gráfica no curso?\n2. Qual é o objetivo principal do componente curricular de Computação Gráfica?\n3. Quais são as referências básicas indicadas para o estudo da Computação Gráfica?\n4. Em que editora e ISBN pode ser encontrado o livro "Computer Graphics: Principles and Practice" de Hughes et al., uma das referências fornecidas?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1732, 'total_tokens': 1852, 'completion_tokens': 120}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-446c7af0-c096-446b-968d-0dc0e919d95d-0' usage_metadata={'input_tokens': 1732, 'output_tokens': 120, 'total_tokens': 1852}


100%|██████████| 207/207 [15:32<00:00,  4.50s/it]

4
content='1. Quais são as etapas da avaliação semestral realizada pelo Colegiado do Curso de Computação sobre os componentes curriculares?\n2. Como os resultados da avaliação semestral do Colegiado do Curso de Computação contribuem para a reflexão do processo ensino-aprendizagem?\n3. De que maneira os dados coletados na autoavaluação semestral são utilizados para subsidiar o alcance dos objetivos do curso de Ciência da Computação?\n4. Além das avaliações institucionais, o que o Colegiado do Curso de Computação avalia periodicamente?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1143, 'total_tokens': 1305, 'completion_tokens': 162}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-a42a63eb-0758-4268-b770-77aa67ab7f74-0' usage_metadata={'input_tokens': 1143, 'output_tokens': 162, 'total_tokens': 1305}


We'll save each dataset for use later.

> NOTE: These datasets will be provided in the repository in case you run into any issues with the data generation steps or you wish to save API calls.

In [ ]:
len(training_questions)

2012

In [ ]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [ ]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [ ]:
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

In [ ]:
paths = [save_dict_to_json_on_drive(train_dataset, '/content/drive/MyDrive/tcc/preprocessing/ft', 'train_dataset.json'),
save_dict_to_json_on_drive(val_dataset, '/content/drive/MyDrive/tcc/preprocessing/ft', 'val_dataset.json'),
save_dict_to_json_on_drive(test_dataset, '/content/drive/MyDrive/tcc/preprocessing/ft', 'test_dataset.json')]

{'questions': {'85a2a905-e015-4e62-8531-85074cea02fe': 'Em quantos estados brasileiros a Universidade Federal da Fronteira Sul está localizada?', '1cb2c8bd-abff-4362-a3a0-3489a4095813': 'Qual é a cidade sede da universidade?', 'df57b81c-500b-48d0-9064-f1db5e656c6a': 'Quantos *campi* possui a universidade em cada um dos estados onde está presente?', '1bebe316-b2a4-4955-9b3d-9751314f8987': 'Em quais cidades, além da sede, a universidade possui *campi* no estado de Santa Catarina?', '774c39d7-280a-4b85-b3ed-451f3bf66d64': 'Quem é o Pro-Reitor responsável pelo setor de Graduação no nosso curso?', 'f57fe6f1-0bd4-4ee7-add0-51e7d31b9872': 'Além da grade obrigatória, o curso oferece disciplinas optativas? Se sim, quais são as áreas abrangidas por essas disciplinas?', '9af8382c-1200-4bfe-9f3d-3b2f836021cf': 'Como o processo de avaliação dos alunos é conduzido no curso de Ciência da Computação?', '2bf408ce-0410-47e3-99a1-066af640ea60': 'Existe algum procedimento específico para realização do Tra

In [ ]:

['/content/drive/MyDrive/tcc/preprocessing/ft/538ac8_train_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/538ac8_val_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/538ac8_test_dataset.json']

In [ ]:

['/content/drive/MyDrive/tcc/preprocessing/ft/538ac8_train_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/538ac8_val_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/538ac8_test_dataset.json']

['/content/drive/MyDrive/tcc/preprocessing/ft/e1c9fc_train_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/e1c9fc_val_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/e1c9fc_test_dataset.json']

## Task 4: Fine-tuning `snowflake-arctic-embed-m`

Now that we have a dataset, let's grab a `sentence-transformers` Embeddings model!

We'll be using Snowflake's [`snowflake-arctic-embed-m`](https://huggingface.co/Snowflake/snowflake-arctic-embed-m) as a base embeddings model.

It is a well performing embeddings model by itself, but there's a lot of very specific domain terms and vocabulary in our courpus - so lets fine-tune it and see what that can do for us!

In [ ]:
# prompt: load the files within paths

import json

def load_json_from_drive(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return data
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {file_path}")
        return None


paths = ['/content/drive/MyDrive/tcc/preprocessing/ft/538ac8_train_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/538ac8_val_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/538ac8_test_dataset.json']

loaded_data = []
for path in paths:
  loaded_data.append(load_json_from_drive(path))

train_dataset = loaded_data[0]
val_dataset = loaded_data[1]
test_dataset = loaded_data[2]

In [ ]:
!pip install -qU sentence_transformers

In [ ]:
!pip install -qU datasets pyarrow

In [ ]:
from sentence_transformers import SentenceTransformer

model_id = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
model = SentenceTransformer(model_id)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


We'll grab some necessary imports from `sentence_transformers` and `torch`.

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

We're using a toy batch size here to reflect the limited number of examples we have.

Ideally we'd want to use a much larger batch size. (~64+)

In [ ]:
BATCH_SIZE = 10

Let's move our dataset into the expected format for training.

In [ ]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

Now we can create a `torch` `DataLoader`!

In [ ]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

Next up, we'll prepare our loss function!

The loss we're using today is called `MultipleNegativesRankingLoss` - you can find more information [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MultipleNegativesRankingLoss.py).

In [ ]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

Now we can set-up our evaluator.

> NOTE: Due to the formatting of our dataset - this is all we have to do!

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

We'll train this model for 5 epochs, though you could increase this number if we had a significant amount more data.

In [ ]:
EPOCHS = 30

In [ ]:
FT_MODEL_OUTPUT_PATH = 'multi-qa-mpnet-base-dot-v1-finetuned-for-ppc-v1'

It's training time!

> NOTE: We're manually defining a warm-up period here - this is just to provide a smooth ramp into our training!

In [ ]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path=FT_MODEL_OUTPUT_PATH,
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=200,
)

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100,Dot Accuracy@1,Dot Accuracy@3,Dot Accuracy@5,Dot Accuracy@10,Dot Precision@1,Dot Precision@3,Dot Precision@5,Dot Precision@10,Dot Recall@1,Dot Recall@3,Dot Recall@5,Dot Recall@10,Dot Ndcg@10,Dot Mrr@10,Dot Map@100
200,No log,No log,0.526961,0.703431,0.769608,0.846814,0.526961,0.234477,0.153922,0.084681,0.526961,0.703431,0.769608,0.846814,0.682077,0.629918,0.635986,0.523284,0.703431,0.762255,0.839461,0.523284,0.234477,0.152451,0.083946,0.523284,0.703431,0.762255,0.839461,0.678101,0.626867,0.633244
202,No log,No log,0.533088,0.702206,0.770833,0.846814,0.533088,0.234069,0.154167,0.084681,0.533088,0.702206,0.770833,0.846814,0.684986,0.633765,0.639934,0.529412,0.704657,0.761029,0.841912,0.529412,0.234886,0.152206,0.084191,0.529412,0.704657,0.761029,0.841912,0.681083,0.630184,0.636423
400,No log,No log,0.553922,0.737745,0.817402,0.890931,0.553922,0.245915,0.163480,0.089093,0.553922,0.737745,0.817402,0.890931,0.718804,0.664011,0.668575,0.551471,0.740196,0.812500,0.889706,0.551471,0.246732,0.162500,0.088971,0.551471,0.740196,0.812500,0.889706,0.716839,0.661760,0.666417
404,No log,No log,0.550245,0.745098,0.817402,0.886029,0.550245,0.248366,0.163480,0.088603,0.550245,0.745098,0.817402,0.886029,0.716269,0.662073,0.667025,0.553922,0.743873,0.813725,0.888480,0.553922,0.247958,0.162745,0.088848,0.553922,0.743873,0.813725,0.888480,0.717386,0.662953,0.667702
600,2.622200,No log,0.575980,0.770833,0.846814,0.915441,0.575980,0.256944,0.169363,0.091544,0.575980,0.770833,0.846814,0.915441,0.745478,0.691066,0.694287,0.572304,0.767157,0.843137,0.916667,0.572304,0.255719,0.168627,0.091667,0.572304,0.767157,0.843137,0.916667,0.744397,0.689299,0.692428
606,2.622200,No log,0.567402,0.767157,0.839461,0.909314,0.567402,0.255719,0.167892,0.090931,0.567402,0.767157,0.839461,0.909314,0.737861,0.682974,0.686422,0.562500,0.770833,0.837010,0.906863,0.562500,0.256944,0.167402,0.090686,0.562500,0.770833,0.837010,0.906863,0.735417,0.680339,0.683964
800,2.622200,No log,0.589461,0.786765,0.839461,0.894608,0.589461,0.262255,0.167892,0.089461,0.589461,0.786765,0.839461,0.894608,0.745223,0.696925,0.701642,0.588235,0.785539,0.839461,0.897059,0.588235,0.261846,0.167892,0.089706,0.588235,0.785539,0.839461,0.897059,0.744461,0.695312,0.699827
808,2.622200,No log,0.596814,0.785539,0.850490,0.903186,0.596814,0.261846,0.170098,0.090319,0.596814,0.785539,0.850490,0.903186,0.751376,0.702441,0.706409,0.594363,0.786765,0.852941,0.900735,0.594363,0.262255,0.170588,0.090074,0.594363,0.786765,0.852941,0.900735,0.749784,0.701019,0.705220
1000,0.598100,No log,0.625000,0.801471,0.863971,0.922794,0.625000,0.267157,0.172794,0.092279,0.625000,0.801471,0.863971,0.922794,0.774627,0.727068,0.730138,0.627451,0.799020,0.870098,0.920343,0.627451,0.266340,0.174020,0.092034,0.627451,0.799020,0.870098,0.920343,0.773975,0.726940,0.730202
1010,0.598100,No log,0.616422,0.802696,0.862745,0.922794,0.616422,0.267565,0.172549,0.092279,0.616422,0.802696,0.862745,0.922794,0.770231,0.721235,0.724002,0.616422,0.801471,0.861520,0.922794,0.616422,0.267157,0.172304,0.092279,0.616422,0.801471,0.861520,0.922794,0.769009,0.719744,0.722492


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [ ]:
import shutil

caminho_colab = FT_MODEL_OUTPUT_PATH

# Caminho para onde você deseja salvar no Google Drive
caminho_drive = '/content/drive/My Drive/tcc/ft/teste3'

# Copiar a pasta do Colab para o Google Drive
shutil.copytree(caminho_colab, caminho_drive)


'/content/drive/My Drive/tcc/ft/teste3'

## Task 5: Evaluating our Retriever

Now that we have fine-tuned our retriever - let's see if it's worthwhile!

We'll start with some basic imports.

In [ ]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

Now we'll define a function that will help us evaluate our retrieval process.

> NOTE: We're assuming 1 correct document in a "hit".

In [ ]:
import tqdm

def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm.tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

All that's left to do is evaluate, we'll evaluate our model against:

1. OpenAI's closed source `text-embedding-3-small`
2. The base non-fine-tuned version of `Snowflake/snowflake-arctic-embed-m`.

Let's see how it stacks up!

### `text-embedding-3-small`

In [ ]:
te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 100/100 [00:19<00:00,  5.25it/s]


In [ ]:
te3_results_df = pd.DataFrame(te3_results)

In [ ]:
te3_hit_rate = te3_results_df["is_hit"].mean()
te3_hit_rate

1.0

### `Snowflake/snowflake-arctic-embed-m` (base)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 820/820 [00:11<00:00, 74.01it/s]


In [ ]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [ ]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

0.5951219512195122

### `Snowflake/snowflake-arctic-embed-m` (fine-tuned)

In [ ]:
finetune_embeddings = HuggingFaceEmbeddings(model_name=FT_MODEL_OUTPUT_PATH)
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

100%|██████████| 820/820 [00:12<00:00, 65.79it/s]


In [ ]:
finetune_results_df = pd.DataFrame(finetune_results)

In [ ]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

0.8609756097560975

## SCommitting to HF Repo

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import HfApi, Repository
from google.colab import userdata
# Define repo name and directory
repo_name = "cc-uffs-ppc-ft-test-multiqa"
local_model_dir = FT_MODEL_OUTPUT_PATH

# Create or push to the repo
api = HfApi()
repo_url = api.create_repo(repo_name, token = settings.HF_TOKEN, private=False, exist_ok=True)

# Use HfApi to upload files directly
api.upload_folder(
    token = settings.HF_TOKEN,
    folder_path=local_model_dir,
    path_in_repo="",
    repo_id=f"winderfeld/{repo_name}",
    repo_type="model"
)




model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/winderfeld/cc-uffs-ppc-ft-test-multiqa/commit/a6f6b89621aea8596889028b20e956eca48ea75d', commit_message='Upload folder using huggingface_hub', commit_description='', oid='a6f6b89621aea8596889028b20e956eca48ea75d', pr_url=None, pr_revision=None, pr_num=None)

## Vibe Checking the RAG Pipeline

We're going to use our RAG pipeline to vibe check on some common phrases now that we've modified it!

### Creating New Chunks

In order to try and evaluate our system more fairly, let's create new chunks that we will use to create our Vector Store.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 50,
    length_function = len
)

training_documents = text_splitter.split_documents(training_documents_loaded.load())

### Base Chain

We'll start by constructing our base chain, which will use the untrained retrieval model.

#### R - Retrieval

In [ ]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

#### A - Augmented

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

#### G - Generation

In [ ]:
rag_llm =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

#### RAG - LCEL RAG Pipeline

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [ ]:
base_rag_chain.invoke({"question" : "What is Ayahuasca?"})["response"]

'I do not know.'

In [ ]:
base_rag_chain.invoke({"question" : "What is the difference between Neurosurgery vs. Neuralink surgery?"})["response"]

'The difference between neurosurgery and Neuralink surgery lies primarily in the approach and risk involved. Traditional neurosurgery often involves more invasive procedures, such as opening deeper parts of the brain or manipulating blood vessels, which carry significant risks. In contrast, Neuralink surgery involves cortical micro-insertions that are performed on the surface of the brain, which significantly reduces the risk compared to more invasive surgeries like those for tumors or aneurysms. Additionally, Neuralink aims to utilize advanced technology and robotics to enhance precision in electrode placement, potentially changing the landscape of neurosurgical practices.'

In [ ]:
base_rag_chain.invoke({"question" : "What is Neural Dust?"})["response"]

'I do not know.'

In [ ]:
base_rag_chain.invoke({"question" : "What is a Neural Decoder?"})["response"]

'A Neural Decoder is a system or algorithm that interprets neural signals, such as sequences of spikes from neurons, to extract meaningful information or predictions. It involves machine learning techniques to create a mapping between the neural data and specific outputs or labels, addressing challenges in architecture and hyperparameters to optimize performance.'

In [ ]:
base_rag_chain.invoke({"question" : "Who got Neuralink surgery?"})["response"]

'The first human being to receive Neuralink surgery is referred to as Noland.'

### Fine-tuned Embedding Model

Now let's rebuild our RAG chain with the Fine-tuned model - the only component we need to change is our `FAISS` vectorstore!

In [ ]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [ ]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [ ]:
finetune_rag_chain.invoke({"question" : "What is Ayahuasca?"})["response"]

'Ayahuasca is a psychoactive brew made from the Banisteriopsis caapi vine and other ingredients, often used in traditional Amazonian shamanic practices for spiritual and healing purposes. It contains DMT (dimethyltryptamine), a powerful hallucinogenic compound, which can induce altered states of consciousness and profound experiences.'

In [ ]:
finetune_rag_chain.invoke({"question" : "What is the difference between Neurosurgery vs. Neuralink surgery?"})["response"]

'Neurosurgery typically involves more complex procedures that may include opening deeper parts of the brain or manipulating blood vessels, which carries significant risks. In contrast, Neuralink surgery focuses on making cortical micro-insertions on the surface of the brain, which is considered to carry significantly less risk compared to traditional neurosurgeries like tumor or aneurysm surgeries. Additionally, Neuralink has developed a rigorous practice regimen, including hundreds of animal surgeries and the use of lifelike models to simulate the procedure before performing it on humans.'

In [ ]:
finetune_rag_chain.invoke({"question" : "What is Neural Dust?"})["response"]

'Neural dust refers to a technology that involves tiny, wireless sensors that can be implanted in the brain to monitor neural activity. These sensors are designed to communicate with external devices, allowing for real-time data collection and potentially enabling advanced brain-computer interfaces.'

In [ ]:
finetune_rag_chain.invoke({"question" : "What is a Neural Decoder?"})["response"]

'A Neural Decoder is a system or model that interprets and translates neural signals, such as sequences of spikes from neurons, into meaningful outputs or predictions. It involves building a dataset, optimizing labels for the model, and determining the appropriate architecture and hyperparameters to effectively map the input data to the desired output. The process combines elements of machine learning, science, and art to create a reliable and efficient decoding mechanism.'

In [ ]:
finetune_rag_chain.invoke({"question" : "Who got Neuralink surgery?"})["response"]

'Noland Arbaugh got Neuralink surgery.'

## RAGAS Evaluation

It's great to have some idea of how our system is doing based on vibe-checks, but let's use RAGAS to provide more insight info. on how things are improving!

In [ ]:
!pip install -qU ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.2 MB/s eta 0:00:00


### RAGAS Synthetic Testset Generation

First things first, we need to generate some data to test our model on.

Let's use our test data that we created before as a base!

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

In [ ]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

In [ ]:
testset = generator.generate_with_langchain_docs(test_split_documents, test_size=20, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/100 [00:00<?, ?it/s]

Generating:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
testset.to_pandas().head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How does the variability of tetraplegia situat...,"[DJ Seo (03:27:14) Yeah, I mean the primary go...",The variability of tetraplegia situations impa...,simple,"[{'source': 'elon_lex_transcript.html.html', '...",True
1,What is the potential future scenario of havin...,"[Lex Fridman (03:37:00) And then, you’d be con...",The potential future scenario is that there co...,simple,"[{'source': 'elon_lex_transcript.html.html', '...",True
2,What was Matthew MacDougall's initial focus wh...,"[Matthew MacDougall (03:45:07) Basically, the ...",Matthew MacDougall's initial focus when lookin...,simple,"[{'source': 'elon_lex_transcript.html.html', '...",True
3,What led the speaker to decide to study the br...,"[Lex Fridman (03:48:25) So, from there, the ea...",The speaker decided to study the brain because...,simple,"[{'source': 'elon_lex_transcript.html.html', '...",True
4,How does being nervous while talking affect pe...,[Lex Fridman (03:26:05) … a lot of novel insig...,Being nervous while talking can affect perform...,simple,"[{'source': 'elon_lex_transcript.html.html', '...",True


### Generating Answer Datasets

For each of our pipelines, let's generate answers to these questions!

Once we have our: Questions, Answers, Contexts, Ground Truths we can move on to evaluating our datasets!

In [ ]:
from datasets import Dataset

def generate_answers(chain, testset):
  answers = []
  contexts = []
  questions = testset.to_pandas()["question"].values.tolist()
  ground_truths = testset.to_pandas()["ground_truth"].values.tolist()

  for question in tqdm.tqdm(questions):
    answer = chain.invoke({"question" : question})
    answers.append(answer["response"])
    contexts.append([context.page_content for context in answer["context"]])

  return Dataset.from_dict({
      "question" : questions,
      "answer" : answers,
      "contexts" : contexts,
      "ground_truth" : ground_truths
  })

In [ ]:
base_dataset = generate_answers(base_rag_chain, testset)

100%|██████████| 20/20 [00:25<00:00,  1.28s/it]


In [ ]:
finetune_dataset = generate_answers(finetune_rag_chain, testset)

100%|██████████| 20/20 [00:29<00:00,  1.48s/it]


### Evaluating Using the Test Set

Now that we have a test set - it's time to evaluate our pipelines with it!

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [ ]:
from ragas import evaluate

result = evaluate(
    base_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
result

{'context_precision': 0.5128, 'faithfulness': 0.4737, 'answer_relevancy': 0.4764, 'context_recall': 0.4583}

In [ ]:
result.to_pandas().head()

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,How does the variability of tetraplegia situat...,I do not know.,"[Future Neuralink capabilities, that you can e...",The variability of tetraplegia situations impa...,1.0,0.0,0.000000,0.0
1,What is the potential future scenario of havin...,I do not know.,"[Future Neuralink capabilities, experience, I’...",The potential future scenario is that there co...,0.0,0.0,0.000000,0.0
2,What was Matthew MacDougall's initial focus wh...,Matthew MacDougall's initial focus when lookin...,[Matthew MacDougall (03:48:34) Yeah. It was so...,Matthew MacDougall's initial focus when lookin...,1.0,1.0,0.999999,0.0
3,What led the speaker to decide to study the br...,The speaker was influenced to study the brain ...,"[into delving deep into the human brain, so no...",The speaker decided to study the brain because...,1.0,1.0,0.977114,0.0
4,How does being nervous while talking affect pe...,I do not know.,"[point, muscle memory kicks in and you sort of...",Being nervous while talking can affect perform...,1.0,0.0,0.000000,0.0


In [ ]:
result = evaluate(
    finetune_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
result

{'context_precision': 0.8086, 'faithfulness': 0.8799, 'answer_relevancy': 0.8680, 'context_recall': 0.9667}

In [ ]:
result.to_pandas().head()

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,How does the variability of tetraplegia situat...,The variability of tetraplegia situations impa...,"[DJ Seo (03:27:14) Yeah, I mean the primary go...",The variability of tetraplegia situations impa...,0.966667,1.000000,1.000000,1.0
1,What is the potential future scenario of havin...,The potential future scenario suggests that th...,[Lex Fridman (00:31:41) And that output rate w...,The potential future scenario is that there co...,1.000000,0.800000,0.000000,1.0
2,What was Matthew MacDougall's initial focus wh...,Matthew MacDougall's initial focus when lookin...,[Matthew MacDougall (03:48:34) Yeah. It was so...,Matthew MacDougall's initial focus when lookin...,1.000000,1.000000,1.000000,1.0
3,What led the speaker to decide to study the br...,The speaker was influenced by personal experie...,"[(01:29:17) But also at the same time, I think...",The speaker decided to study the brain because...,1.000000,1.000000,0.977114,1.0
4,How does being nervous while talking affect pe...,Being nervous while talking can negatively imp...,"[point, muscle memory kicks in and you sort of...",Being nervous while talking can affect perform...,0.700000,0.428571,0.997113,1.0


## Conclusion

As you can see - with only a few hundred data points, we're able to increase our embedding model and increase the effectiveness of RAG!